### Minimum Number of Flips to Convert Binary Matrix to Zero Matrix
Given a m x n binary matrix mat. In one step, you can choose one cell and flip it and all the four neighbours of it if they exist (Flip is changing 1 to 0 and 0 to 1). A pair of cells are called neighboors if they share one edge.

Return the minimum number of steps required to convert mat to a zero matrix or -1 if you cannot.

Binary matrix is a matrix with all cells equal to 0 or 1 only.

Zero matrix is a matrix with all cells equal to 0.

In [1]:
class Solution:
    def minFlips(self, mat) -> int:
        hashcode = 0; row = len(mat); col = len(mat[0])
        for i in range(row):
            for j in range(col):
                if mat[i][j] == 1:
                    k = i*col + j
                    hashcode |= 1<<k
                    
        if hashcode == 0:
            return 0
        
        queue = [hashcode]; seen = {hashcode}
        level = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                hashcode = queue.pop(0)
                if hashcode == 0:
                    return level
                for i in range(row):
                    for j in range(col):
                        new_hashcode = self.switch(i, j, row, col, hashcode)
                        if new_hashcode not in seen:
                            seen.add(new_hashcode)
                            queue.append(new_hashcode)
            level += 1
        return -1
        
    def switch(self, i, j, r, c, hashcode):
        for x, y in [(i,j), (i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            k = x*c + y
            if 0<=x<r and 0<=y<c:
                hashcode ^= 1<<k
        
        return hashcode

grid = [[1,1,1],[1,0,1],[0,0,0]]
Solution().minFlips(grid)

6

###  Shortest Path Visiting All Nodes
An undirected, connected graph of N nodes (labeled 0, 1, 2, ..., N-1) is given as graph.

graph.length = N, and j != i is in the list graph[i] exactly once, if and only if nodes i and j are connected.

Return the length of the shortest path that visits every node. You may start and stop at any node, you may revisit nodes multiple times, and you may reuse edges.

In [2]:
def shortestPathLength(graph) -> int:
    queue = []; seen = set()
    num_nodes = len(graph)
    for i in range(num_nodes):
        queue.append((i, 1<<i))
        seen.add((i, 1<<i))

    level = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            node, hashcode = queue.pop(0)
            if hashcode == 2**num_nodes-1:
                return level
            for i in graph[node]:
                if (i, hashcode | 1<<i) not in seen:
                    seen.add((i, hashcode|1<<i))
                    queue.append((i, hashcode|1<<i))
        level += 1
    
graph = [[1],[0,2,4],[1,3,4],[2],[1,2]]
shortestPathLength(graph)

4

### Bus Routes
We have a list of bus routes. Each routes[i] is a bus route that the i-th bus repeats forever. For example if routes[0] = [1, 5, 7], this means that the first bus (0-th indexed) travels in the sequence 1->5->7->1->5->7->1->... forever.

We start at bus stop S (initially not on a bus), and we want to go to bus stop T. Travelling by buses only, what is the least number of buses we must take to reach our destination? Return -1 if it is not possible.

In [10]:
from collections import defaultdict
class Solution:
    def numBusesToDestination(self, routes, S: int, T: int) -> int:
        if S == T: return 0
        hashmap = defaultdict(set)
        queue = []; seen = set()
        for route_index, route in enumerate(routes):
            for stop in route:
                hashmap[stop].add(route_index)
        
        for route_index in hashmap[S]:
            queue.append(route_index)
            seen.add(route_index)
        
        num_buses = 1
        while queue:
            size = len(queue)
            for _ in range(size):
                route_index = queue.pop(0)
                if route_index in hashmap[T]:
                    return num_buses
                for stop in routes[route_index]:
                    for next_index in hashmap[stop]:
                        if next_index not in seen:
                            seen.add(next_index)
                            queue.append(next_index)
            num_buses += 1
        return -1

routes = [[1, 2, 7], [3, 6, 7]]
S = 1
T = 6
Solution().numBusesToDestination(routes, S, T)

2